In [ ]:
"""
Created on Wed Oct 10 15:00:44 2018

@author: leon.zhao
"""

import pandas as pd
import plotly.plotly as ply
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import warnings
import itertools
import statsmodels.api as sm

#from statsmodels.tsa.arima_model import ARIMA
#from sklearn.metrics import mean_squared_error
#from math import sqrt


ply.sign_in("leon.zhao", "X91E8tOmjTVzl7G5ePOu")

df2 = pd.read_csv("/Users/leon.zhao/Desktop/py/daily_metric.csv" ,index_col=0)

df2.head()
df2.index = pd.to_datetime(df2.index)
df3 = df2.loc['2018',['ITEMS_POSTED']]
#df3 = df2.loc['2017':'2018',['ITEMS_POSTED']]
from numpy import log10
from pandas import DataFrame
df4 = DataFrame(log10(df3['ITEMS_POSTED']))


fig = go.Scatter( 
    x = df2.index.tolist(),
    y = df2['ITEMS_POSTED'],
    mode = 'lines',
    name = 'lines',
    line = dict( 
        color = ('rgb(205, 12, 24)'),
#        width = '5',
#        colorscale = 'Viridis',
#        showscale = True 
    )
)

layout = dict(
            title = "Item Posted 2017 -- 2018",
            xaxis = dict(title = "Date"),
            yaxis = dict(title = "Items Posted"),
            )
        
graph1 = dict(data = [fig], layout = layout)

ply.iplot(graph1, filename = 'Item Posted With style')

from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df4, model='multiplicative')
seasonal_decompose = result.plot()
plot_mpl(seasonal_decompose, resize=True)


# Define the p, d and q parameters to take any value between
p = d = q = range(3, 5)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[1], x[1], x[1], 7) for x in list(itertools.product(p, d, q))]

warnings.filterwarnings("ignore") # specify to ignore warning messages

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(df4,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{} BIC:{}'.format(param, param_seasonal, results.aic, results.bic))
        except:
            continue

mod = sm.tsa.statespace.SARIMAX(df4,
                                order=(3, 3, 4),
                                seasonal_order=(4, 4, 4, 7),
                                enforce_stationarity=False,
                                enforce_invertibility=False)


results = mod.fit()
yhat = results.forecast()
print(results.aic)
print(results.bic)
print(results.resid)
print(results.summary())
print(yhat)

	
yhat = results.predict(start=pd.to_datetime('2018-10-01'), end=pd.to_datetime('2018-12-31'))
print(yhat)

model_diagnostics = results.plot_diagnostics(figsize=(15, 12))
plt.show()

df5 = DataFrame(yhat)
df5.columns = ['ITEMS_POSTED_FORECASTED']

df5.to_csv('df5', encoding='utf-8', index=True)


writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')
df5.to_excel(writer, sheet_name='Sheet1')
writer.save()


from pandas.tools.plotting import autocorrelation_plot
autocorrelation_plot(df4)

from statsmodels.tsa.stattools import adfuller
result = adfuller(df5['ITEMS_POSTED_FORECASTED'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

residuals = results.resid
residuals.plot()
residuals.plot(kind='kde')
print(residuals.describe())



pred = results.get_prediction(start=pd.to_datetime('2018-09-01'), end=pd.to_datetime('2018-11-01'), dynamic=False)
pred_ci = pred.conf_int()

ax = df4['2017':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('DATE')
ax.set_ylabel('ITEMS_POSTED')
plt.legend()

plt.show()


